In [3]:
import cotengra as ctg
from _formula import *
from _sat import *

In [10]:
## Rank-1 tensor for output nodes.
output_t: np.ndarray = np.array([0, 1])

## Rank-2 tensor for negation nodes.
negation_t: np.ndarray = np.array([[0, 1], [1, 0]])

## Rank-3 tensors for binary operations of conjunction and disjunction.
conj_t: np.ndarray = np.zeros((2, 2, 2))

for i in range(2):
    for j in range(2):
        for k in range(2):
            conj_t[i][j][k] = int(int(i and j) == int(k))

The `input_arrays` function constructs the tensors that the tensor network representation of the CNF will be instantiated with to perform the contraction.

In [27]:
def input_arrays(cnf: CNFInstance) -> List[np.ndarray]:
    inputs = cnf.network().wiring.hypergraph.inputs
    arrays = []

    for n in inputs:
        if len(n) == 1:
            arrays.append(output_t)
        elif len(n) == 2:
            arrays.append(negation_t)
        elif len(n) == 3:
            arrays.append(conj_t)

    return arrays

In [48]:
def einsum_expression(cnf: CNFInstance):
    graph = cnf.network().wiring.hypergraph
    expr = ctg.utils.inputs_output_to_eq(graph.inputs, graph.output)

    [inputs, outputs] = expr.split("->")

    n = max([ int(n) for n in inputs if n != "," ])

    for i in range(n+1):
        inputs = inputs.replace(str(i), str(chr(97 + i)))
        outputs = outputs.replace(str(i), str(chr(97 + i)))

    return inputs + "->" + outputs

In [49]:
cnf1: CNFInstance = CNFInstance.random(k=3, n=3, m=2, rng=0)
cnf2: CNFInstance = CNFInstance.from_formula(neg(conj(disj(variable("a"), variable("b")), conj(neg(variable("a")), neg(variable("b"))))))

expr1: str = einsum_expression(cnf1)
expr2: str = einsum_expression(cnf2)

print(expr1)

abd,dce,e,cf,bg,ah,fgi,ihj,j->abc


In [50]:
ctg.einsum(expr2, *input_arrays(cnf2))

array([[0., 0.],
       [0., 0.]])